# Moderation Analysis: Individual Differences in Response to AI Uncertainty

## Research Question

While our main hypothesis testing showed no significant main effect of uncertainty communication on trust in AI systems, **moderation analysis** asks a more nuanced question:

> **For whom or under what conditions does uncertainty communication affect trust?**

Even when an overall effect is null, different subgroups may respond differently to the intervention:
- High-tech-affinity individuals might trust the AI *more* when shown uncertainty (transparency valued)
- Low-tech-affinity individuals might trust it *less* (uncertainty perceived as incompetence)
- These opposing effects could cancel out, producing the observed null main effect

## Statistical Framework

### Moderation Analysis

We test moderation using multiple regression with interaction terms:

$$Y = \beta_0 + \beta_1 X + \beta_2 M + \beta_3 (X \times M) + \epsilon$$

Where:
- $Y$ = outcome (TiA subscale)
- $X$ = treatment (uncertainty vs control)
- $M$ = moderator (e.g., age, ATI)
- $\beta_3$ = **moderation effect** (interaction)
- $\beta_2$ = **direct effect** of moderator

### What We Extract

For each moderator-outcome pair, we extract TWO effects:

1. **Moderation Effect (β₃)**: Does the moderator change how uncertainty affects trust?
2. **Direct Effect (β₂)**: Does the moderator predict trust, regardless of experimental condition?

## Analysis Approach

- **Unified Analysis**: All moderators (theoretical + exploratory) analyzed together
- **Hypothesis Type Flagged**: Results marked as "theoretical" or "exploratory"
- **Interpretations Included**: Human-readable interpretations saved to CSV
- **Multiple Outputs**: Separate CSVs for moderation effects, direct effects, and correlation matrix

## Setup and Data Loading

In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.multitest import multipletests
from scripts.stats import (test_moderation, interpret_moderation, 
                           interpret_direct_effect, format_effect_with_stars)
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.precision', 4)
pd.set_option('display.max_columns', None)

# Define variable names
tia_scales = ['tia_f', 'tia_pro', 'tia_rc', 'tia_t', 'tia_up']
scale_titles = {
    'tia_rc': 'TiA - Reliability/Confidence',
    'tia_up': 'TiA - Understanding/Predictability',
    'tia_f': 'TiA - Familiarity',
    'tia_pro': 'TiA - Propensity to Trust',
    'tia_t': 'TiA - Trust in Automation'
}

In [2]:
# Load data
data = pd.read_csv('../data/data_scales.csv')
print(f"Total sample size: {len(data)}")
print(f"\nGroup distribution:")
print(data['stimulus_group'].value_counts())

Total sample size: 255

Group distribution:
stimulus_group
1    129
0    126
Name: count, dtype: int64


## Data Preparation

### Variable Centering and Coding

We prepare variables for moderation analysis:

1. **Effect code treatment**: stimulus_group as -0.5 (control) and 0.5 (uncertainty)
2. **Mean-center continuous variables**: For interpretability and to reduce multicollinearity
3. **Effect code categorical variables**: For symmetric interpretation

**Why mean-center?**
- Main effects interpretable at average moderator level
- Reduces multicollinearity
- Doesn't change interaction coefficient

In [3]:
# Create working copy
df = data.copy()

# 1. Effect code treatment: control = -0.5, uncertainty = 0.5
df['group_effect'] = df['stimulus_group'] - 0.5

# 2. Mean-center all continuous moderators
continuous_vars = ['hcsds_c', 'hcsds_v', 'ati', 'age',
                  'manip_check1_1', 'manip_check1_2', 'manip_check1_3', 'manip_check1_4',
                  'page_submit']

for var in continuous_vars:
    df[f'{var}_c'] = df[var] - df[var].mean()

# 3. Effect code gender (binary)
gender_mapping = df['gender'].value_counts().index[:2]  # Two most common
df_full = df.copy()  # Keep full dataset
df = df[df['gender'].isin(gender_mapping)].copy()  # Filter for gender analysis
df['gender_c'] = df['gender'].map({gender_mapping[0]: -0.5, gender_mapping[1]: 0.5})

# 4. Mean-center ordinal variables (education, AI experience)
df['education_c'] = df['education'] - df['education'].mean()
df['Q19_c'] = df['Q19'] - df['Q19'].mean()

print(f"Prepared {len(df)} observations for analysis")
print(f"Continuous moderators: {len(continuous_vars)}")
print(f"Total moderators to test: {len(continuous_vars) + 3}")  # + gender, education, Q19

Prepared 250 observations for analysis
Continuous moderators: 9
Total moderators to test: 12


## Define All Moderators

We specify all moderators to analyze, flagging each as theoretical (a priori hypothesis) or exploratory (post-hoc).

In [4]:
# Define all moderators with their properties
moderators = [
    # Theoretical moderators (a priori hypotheses)
    {'name': 'ATI', 'variable': 'ati_c', 'hypothesis_type': 'theoretical'},
    {'name': 'Healthcare Trust - Competence', 'variable': 'hcsds_c_c', 'hypothesis_type': 'theoretical'},
    {'name': 'Healthcare Trust - Values', 'variable': 'hcsds_v_c', 'hypothesis_type': 'theoretical'},
    {'name': 'Age', 'variable': 'age_c', 'hypothesis_type': 'theoretical'},
    {'name': 'Gender', 'variable': 'gender_c', 'hypothesis_type': 'theoretical'},
    {'name': 'Education', 'variable': 'education_c', 'hypothesis_type': 'theoretical'},
    {'name': 'AI Experience', 'variable': 'Q19_c', 'hypothesis_type': 'theoretical'},
    
    # Exploratory moderators (post-hoc exploration)
    {'name': 'Manipulation Check 1', 'variable': 'manip_check1_1_c', 'hypothesis_type': 'exploratory'},
    {'name': 'Manipulation Check 2', 'variable': 'manip_check1_2_c', 'hypothesis_type': 'exploratory'},
    {'name': 'Manipulation Check 3', 'variable': 'manip_check1_3_c', 'hypothesis_type': 'exploratory'},
    {'name': 'Manipulation Check 4', 'variable': 'manip_check1_4_c', 'hypothesis_type': 'exploratory'},
    {'name': 'Page Submit Time', 'variable': 'page_submit_c', 'hypothesis_type': 'exploratory'},
]

print(f"Total moderators: {len(moderators)}")
print(f"  Theoretical: {sum(1 for m in moderators if m['hypothesis_type'] == 'theoretical')}")
print(f"  Exploratory: {sum(1 for m in moderators if m['hypothesis_type'] == 'exploratory')}")
print(f"\nTotal tests: {len(moderators) * len(tia_scales)} (moderators × outcomes)")

Total moderators: 12
  Theoretical: 7
  Exploratory: 5

Total tests: 60 (moderators × outcomes)


## Unified Moderation Analysis

We analyze all moderators in a single loop, extracting both moderation effects and direct effects simultaneously.

In [5]:
# Storage for results
moderation_results = []
direct_effect_results = []

print("="*80)
print("RUNNING MODERATION ANALYSIS")
print("="*80)
print(f"Testing {len(moderators)} moderators × {len(tia_scales)} outcomes = {len(moderators) * len(tia_scales)} tests\n")

# Main analysis loop
for mod_info in moderators:
    mod_name = mod_info['name']
    mod_var = mod_info['variable']
    hyp_type = mod_info['hypothesis_type']
    
    print(f"Testing moderator: {mod_name} ({hyp_type})")
    
    for outcome in tia_scales:
        # Run moderation analysis
        results, model = test_moderation(df, outcome, mod_name, mod_var)
        
        # Extract moderation effect (interaction)
        moderation_results.append({
            'moderator': mod_name,
            'outcome': outcome,
            'moderated_variable': 'group',
            'hypothesis_type': hyp_type,
            'beta': results['b_interaction'],
            'se': results['se_interaction'],
            'ci_lower': results['ci_interaction_lower'],
            'ci_upper': results['ci_interaction_upper'],
            'p': results['p_interaction'],
            'r_squared': results['r_squared'],
            'adj_r_squared': results['adj_r_squared'],
            'n': results['n'],
            'sig': '***' if results['p_interaction'] < 0.001 else '**' if results['p_interaction'] < 0.01 else '*' if results['p_interaction'] < 0.05 else 'ns',
            'interpretation': interpret_moderation(results['b_interaction'], results['p_interaction'], mod_name)
        })
        
        # Extract direct effect (main effect of moderator)
        direct_effect_results.append({
            'predictor': mod_name,
            'outcome': outcome,
            'hypothesis_type': hyp_type,
            'beta': results['b_moderator'],
            'se': results['se_moderator'],
            'p': results['p_moderator'],
            'r_squared': results['r_squared'],
            'adj_r_squared': results['adj_r_squared'],
            'n': results['n'],
            'sig': '***' if results['p_moderator'] < 0.001 else '**' if results['p_moderator'] < 0.01 else '*' if results['p_moderator'] < 0.05 else 'ns',
            'interpretation': interpret_direct_effect(results['b_moderator'], results['p_moderator'], mod_name, outcome)
        })

print(f"\nCompleted {len(moderation_results)} tests")
print(f"Extracted {len(moderation_results)} moderation effects")
print(f"Extracted {len(direct_effect_results)} direct effects")

RUNNING MODERATION ANALYSIS
Testing 12 moderators × 5 outcomes = 60 tests

Testing moderator: ATI (theoretical)
Testing moderator: Healthcare Trust - Competence (theoretical)
Testing moderator: Healthcare Trust - Values (theoretical)
Testing moderator: Age (theoretical)
Testing moderator: Gender (theoretical)
Testing moderator: Education (theoretical)
Testing moderator: AI Experience (theoretical)
Testing moderator: Manipulation Check 1 (exploratory)
Testing moderator: Manipulation Check 2 (exploratory)
Testing moderator: Manipulation Check 3 (exploratory)
Testing moderator: Manipulation Check 4 (exploratory)
Testing moderator: Page Submit Time (exploratory)

Completed 60 tests
Extracted 60 moderation effects
Extracted 60 direct effects


## Results Summary

### Significant Moderation Effects

In [8]:
# Convert to DataFrames
mod_df = pd.DataFrame(moderation_results)
dir_df = pd.DataFrame(direct_effect_results)

# Filter significant moderation effects
sig_mod = mod_df[mod_df['p'] < 0.05].copy()

print("="*80)
print(f"SIGNIFICANT MODERATION EFFECTS (p < .05): {len(sig_mod)}/{len(mod_df)}")
print("="*80)

if len(sig_mod) > 0:
    sig_mod_display = sig_mod[['moderator', 'outcome', 'beta', 'p', 'sig', 'hypothesis_type', 'interpretation']].copy()
    print(sig_mod_display.to_string(index=False))
else:
    print("No significant moderation effects found.")

# Count by hypothesis type
print(f"\n{'='*100}")
print("Breakdown by hypothesis type:")
for hyp_type in ['theoretical', 'exploratory']:
    n_tested = len(mod_df[mod_df['hypothesis_type'] == hyp_type])
    n_sig = len(sig_mod[sig_mod['hypothesis_type'] == hyp_type])
    print(f"  {hyp_type.capitalize()}: {n_sig}/{n_tested} significant ({n_sig/n_tested*100:.1f}%)")

SIGNIFICANT MODERATION EFFECTS (p < .05): 3/60
           moderator outcome   beta      p sig hypothesis_type                                                                                             interpretation
           Education   tia_t 0.1773 0.0104   *     theoretical            Significant moderation (p = 0.010): The effect of intervention increases as Education increases
Manipulation Check 1  tia_rc 0.1375 0.0216   *     exploratory Significant moderation (p = 0.022): The effect of intervention increases as Manipulation Check 1 increases
    Page Submit Time   tia_t 0.0025 0.0470   *     exploratory     Significant moderation (p = 0.047): The effect of intervention increases as Page Submit Time increases

Breakdown by hypothesis type:
  Theoretical: 1/35 significant (2.9%)
  Exploratory: 2/25 significant (8.0%)


### Significant Direct Effects

In [9]:
# Filter significant direct effects
sig_dir = dir_df[dir_df['p'] < 0.05].copy()

print("="*100)
print(f"SIGNIFICANT DIRECT EFFECTS (p < .05): {len(sig_dir)}/{len(dir_df)}")
print("="*100)

if len(sig_dir) > 0:
    # Group by outcome
    for outcome in tia_scales:
        outcome_effects = sig_dir[sig_dir['outcome'] == outcome]
        if len(outcome_effects) > 0:
            print(f"\n{outcome} ({scale_titles[outcome]}):")
            for _, row in outcome_effects.iterrows():
                print(f"  • {row['predictor']} ({row['hypothesis_type']}): β = {row['beta']:.3f}, p = {row['p']:.4f} {row['sig']}")
                print(f"    {row['interpretation']}")
else:
    print("No significant direct effects found.")

SIGNIFICANT DIRECT EFFECTS (p < .05): 27/60

tia_f (TiA - Familiarity):
  • ATI (theoretical): β = 0.305, p = 0.0000 ***
    Significant positive effect (p < .001): Higher ATI predicts higher tia_f
  • Age (theoretical): β = 0.011, p = 0.0116 *
    Significant positive effect (p = 0.012): Higher Age predicts higher tia_f

tia_pro (TiA - Propensity to Trust):
  • Healthcare Trust - Competence (theoretical): β = 0.122, p = 0.0450 *
    Significant positive effect (p = 0.045): Higher Healthcare Trust - Competence predicts higher tia_pro
  • Healthcare Trust - Values (theoretical): β = 0.194, p = 0.0014 **
    Significant positive effect (p = 0.001): Higher Healthcare Trust - Values predicts higher tia_pro
  • Age (theoretical): β = 0.009, p = 0.0036 **
    Significant positive effect (p = 0.004): Higher Age predicts higher tia_pro
  • Manipulation Check 2 (exploratory): β = 0.085, p = 0.0457 *
    Significant positive effect (p = 0.046): Higher Manipulation Check 2 predicts higher tia_pro

## Effects Matrix

Matrix showing all predictor→outcome relationships with APA-style significance markers.

In [10]:
# Create effects matrix (predictors × outcomes)
matrix_data = {}
for outcome in tia_scales:
    matrix_data[outcome] = {}
    for predictor in dir_df['predictor'].unique():
        row = dir_df[(dir_df['predictor'] == predictor) & (dir_df['outcome'] == outcome)].iloc[0]
        matrix_data[outcome][predictor] = format_effect_with_stars(row['beta'], row['p'])

effects_matrix = pd.DataFrame(matrix_data).T

print("\n" + "="*100)
print("EFFECTS MATRIX: Predictor → Outcome Relationships")
print("="*100)
print("\nRows = Predictors | Columns = TiA Outcomes")
print("Cell values = β coefficient with significance: * p<.05, ** p<.01, *** p<.001\n")
print(effects_matrix.to_string())
print(f"\n{'='*100}\n")


EFFECTS MATRIX: Predictor → Outcome Relationships

Rows = Predictors | Columns = TiA Outcomes
Cell values = β coefficient with significance: * p<.05, ** p<.01, *** p<.001

              ATI Healthcare Trust - Competence Healthcare Trust - Values       Age  Gender Education AI Experience Manipulation Check 1 Manipulation Check 2 Manipulation Check 3 Manipulation Check 4 Page Submit Time
tia_f    0.305***                        -0.071                    -0.033    0.011*   0.164     0.053         0.031               -0.059               -0.029               -0.053               -0.100           -0.000
tia_pro    -0.030                        0.122*                   0.194**   0.009**   0.128    -0.036         0.014               -0.006               0.085*               0.081*             0.161***          -0.001*
tia_rc      0.019                      0.203***                  0.172***  0.011***   0.135     0.027        -0.000               -0.052              0.112**                0.0

## Export Results to CSV

Saving three CSV files:
1. **moderation_effects.csv**: All interaction effects with interpretations
2. **direct_effects.csv**: All predictor→outcome effects with interpretations
3. **effects_matrix.csv**: Matrix of coefficients with significance markers

In [13]:
# Save moderation effects
mod_df.to_csv('../output/moderation_effects.csv', index=False)
print("✓ Saved: output/moderation_effects.csv")
print(f"  - Total effects: {len(mod_df)}")
print(f"  - Significant (p < .05): {(mod_df['p'] < 0.05).sum()}")

# Save direct effects
dir_df.to_csv('../output/direct_effects.csv', index=False)
print("\n✓ Saved: output/direct_effects.csv")
print(f"  - Total effects: {len(dir_df)}")
print(f"  - Significant (p < .05): {(dir_df['p'] < 0.05).sum()}")

# Save effects matrix
effects_matrix.to_csv('../output/effects_matrix.csv')
print("\n✓ Saved: output/effects_matrix.csv")
print(f"  - Dimensions: {effects_matrix.shape[0]} predictors × {effects_matrix.shape[1]} outcomes")

print("\n" + "="*80)
print("ANALYSIS COMPLETE!")
print("="*80)

✓ Saved: output/moderation_effects.csv
  - Total effects: 60
  - Significant (p < .05): 3

✓ Saved: output/direct_effects.csv
  - Total effects: 60
  - Significant (p < .05): 27

✓ Saved: output/effects_matrix.csv
  - Dimensions: 5 predictors × 12 outcomes

ANALYSIS COMPLETE!
